# Isotank Design for the LV4 Sub-Orbital Rocket
## Motivation
With liquid oxygen (LOX) being one of the most common oxidizers used in rocketry, lightweight cryogenic storage are a critical design component. Traditionally metallic materials, such as stainless steel, aluminimun and aluminum-lithium alloys have been partially due to their compatibility with LOX. Composite materials have also been looked at over the years, but finding specific materials that can withstand both the cold temperatures and remain non-reactive can be difficult. As part of the PSAS LV4 program, a capstone team at PSU built a small-scale composite tank design, but there are concerns with scaling up the design to LV4 scales.

In light of the difficulties manufacturing previous designs, a new approach for LV4 LOX tanks using aluminum isogrids is being studied. The goal is to have a design that is manufacturable by the amatuer group, but has a mass close to composite designs.

## Design Choices
### Material
For simplicity, the tank is built from a tube or pipe, eliminating the need to weld the length of the tank. The downside is that this limits the material options to what can be obtained off the shelf. A quick survey of choices found the following options, all made from 6061-T6. Note, material prices as of 2020/10

| ID (in) | Wall (in) | OD (in) | Type | Price/Ft | Source |
|---------|-----------|---------|------|----------|--------|
| 10.02 | 0.365 | 10.75 | Pipe | 270.01 | [Online Metals](https://www.onlinemetals.com/en/buy/aluminum/10-nom-schedule-40-aluminum-pipe-6061-t6-extruded/pid/10619)|
| 10.02 | 0.365 | 10.75 | Pipe | 64.33 | [Midwest Steel & Al*](https://www.midweststeelsupply.com/store/6061aluminumpipe)|
| 10.50 | 0.750 | 12.00 | Tube | 826.71 | [Online Metals](https://www.onlinemetals.com/en/buy/aluminum/12-od-x-0-75-wall-x-10-5-id-aluminum-round-tube-6061-t6-extruded-aluminum-association-in-/pid/21986)|
| 11.00 | 0.500 | 12.00 | Tube | 678.12 | [Metals Depot](https://www.metalsdepot.com/search?q=T3R12500)
| 11.50 | 0.250 | 12.00 | Tube | 181.63 | [Online Metals](https://www.onlinemetals.com/en/buy/aluminum/12-od-x-0-25-wall-x-11-5-id-aluminum-round-tube-6061-t6-extruded/pid/23929)|
| 12.00 | 0.375 | 12.75 | Pipe | 85.10 | [Midwest Steel & Al*](https://www.midweststeelsupply.com/store/6061aluminumpipe)|

*For Midwest Steel and Al, material is considered raw and can have light scratches

Other sizes appear to be available from [Vita Needle](https://www.vitaneedle.com/aluminum-tube/) but quotes have to be requested.

6061-T6 is a an acceptable choice as it has reasonable strength, weldability and LOX compatibility. The 11.5 in inner diameter (ID) pipe was selected as it's closest to the initial LV4 multi-disciplinary design targets and least expensive option.

For initial estimates, a total tank length of 48" is assumed. This fits with available manufacuring possibilities.

In [1]:
# Tank radial dimensions
tank_id = 10.02
tank_wall_t = 0.365
tank_od = tank_id+2*tank_wall_t
tank_overall_length = 48

# Material properties 
E = 69.0 # Young's Modules, GPa for Al 6061-T6
nu = 0.33 # Poisson's Ratio for Al 6061-T6

### Flanges
The use of the flange can increase the strength of the isogrid, but increases machining complexity. It was decided to omit flanges for initil version of the design.


In [2]:
flanges = False

### Skin Thickness
The tank stock material is not highly precise. The part will be machined from the outside in and there is a risk of inadvertantly maching through the pipe if the skin thickness is too small. Additionally, after talking with OMIC, they thought that a skin thickness of 0.05" (50 mil) was reasonable.

In [3]:
skin_t = 0.05


### Rib Depth
The isogrid ribs are the source of the strength of the isogrid design. The depth of the rib is the primary factor is determining the strength. Thefore, for optimimum strength, the rib depth is simply the wall thickness minus the skin thickness 

In [4]:
rib_depth = tank_wall_t - skin_t
print('Rib Depth: {:5.3f}'.format(rib_depth))

### Rib thickness
Again, after discussion with OMIC, it was decided that a rib web thickness of 0.050" (50 mil) could be reasonably achievable.

In [5]:
rib_t = 0.05


### Pattern size
For optimal strength and to maintain isotropic properties, the isogrid cells need to wrap around the tank diameter in integer increments. Twenty four cells was chosen as a starting point.

In [6]:
import math
num_radial_divisions = 24
pattern_size = tank_od*math.pi/num_radial_divisions
print('Pattern Size: {:5.3f}'.format(pattern_size))

Pattern Size: 1.407


## Tank Properties
The isogrid properties are calculated from NASA CR-124075

In [7]:
cell_height = pattern_size
alpha = (rib_t*rib_depth)/(skin_t*cell_height) # Web non-dimensional ratio, pg. 2.0.008
print('Web non-dimensional ratio, Alpha: {:5.3f}'.format(alpha))
delta = rib_depth/skin_t # rib depth to skin thickness ratio, pg. 2.0.008
print('Rib depth to skin thickness, Delta: {:5.3f}'.format(delta))
if flanges == False:
    beta = (3*alpha*(1+delta)**2+(1+alpha)*(1+alpha*delta**2))**0.5
    t_star = skin_t*beta/(1+alpha) # equivilent thickness, Eq. 2.5.3
    t_bar = skin_t*(1+ 3*alpha)
    E_star = E*(1+alpha)**2.0/beta # equivilent Young's Modulus Eq. 2.5.4
print('Beta: {:5.3f}'.format(beta))
print('Equivilant monocoque thickness, t*: {:5.3f} in'.format(t_star))
print('Equivilant weight thickness, t_bar: {:5.3f} in'.format(t_bar))
print('Equivilant monocoque Young\'s Modulus, E*: {:5.3f} GPa'.format(E_star))

Web non-dimensional ratio, Alpha: 0.224
Rib depth to skin thickness, Delta: 6.300
Beta: 6.920
Equivilant monocoque thickness, t*: 0.283 in
Equivilant weight thickness, t_bar: 0.084 in
Equivilant monocoque Young's Modulus, E*: 14.935 GPa


## Mass Model
Equations for calculating the mass of the isotank based on diameter and length

In [8]:
import math

# Ring mass estimate from solidworks model
ring_mass = 0.5316 #kg = 1.172 lbs
ring_circum_density = 0.5316/(math.pi*12.0) # 12.0 since 12 was the diameter for the mass estimate
ring_length = 1.0*0.0254

weld_gap_length = 1.0*0.0254

grid_length = 48*0.0254-2*weld_gap_length-2*ring_length
grid_t_bar = t_bar*0.0254

density = 2700 #kg/m3
t = tank_wall_t*0.0254
id = tank_id*0.0254
ring_mass = 2.0*ring_circum_density*id
weld_gap_mass = 2.0*density*weld_gap_length*(math.pi*((id/2.0 + t)**2-(id/2.0)**2))
tank_wall_mass = density*grid_length*(math.pi*((id/2.0 + grid_t_bar)**2-(id/2.0)**2))

tank_mass = ring_mass + weld_gap_mass + tank_wall_mass

print('tank_mass:',tank_mass, 'kg')

tank_mass: 6.225518665832297 kg


## Load Calculations

### Pressure Only
Max pressure calculation assuming pressure only. Assumes thin-wall and isotropic assumptions


In [9]:
material_name = '6061-T6'
welded_tensile_strength = 24000 # psi
tensile_strength = 40000 # psi
ult_tensile_strength = 45000 # psi

max_pressure = tensile_strength*t_star/(tank_id/2)
print('max pressure for unwelded ,', material_name, ': ',max_pressure, 'psi')
max_welded_pressure = welded_tensile_strength*t_star/(tank_id/2)
print('max pressure for welded ,', material_name, ': ',max_welded_pressure, 'psi')

max pressure for unwelded , 6061-T6 :  2257.159064388755 psi
max pressure for welded , 6061-T6 :  1354.295438633253 psi



### Combined Compression and Bending
For the fuel tank, the cylinder is under compression and bending. Section 4.2 on page 4.2.001 (#97) covers this scenario.